In [1]:
import pandas as pd
import networkx as nx
from gensim.models import Word2Vec
import stellargraph as sg
from stellargraph import StellarGraph
from stellargraph.data import BiasedRandomWalk
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, matthews_corrcoef, confusion_matrix, classification_report
from __future__ import division
import numpy as np
from numpy.linalg import inv
from scipy.spatial.distance import pdist, squareform
import pickle

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


2024-04-20 14:06:38.646010: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-20 14:06:41.365841: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Supporting Functions

In [2]:
# def DSD_calculator(adjacency, walk_length, restart_p):
#     """
#     adjacency - adjacency matrix represented as a numpy array
#                 assumes graph is fully connected.
#     walk_length - the length of random walks used to calculate DSD
#                   if walk_length = -1, then calculate DSD at convergence
#     restart_p - the restart probability
#         if p = 0, then it's a traditional random walk
#     returns DSD matrix represented as a numpy array
#     """
#     adjacency = np.asmatrix(adjacency)
#     n = adjacency.shape[0]
#     degree = adjacency.sum(axis=1)
#     p = adjacency / degree
#     if walk_length >= 0:
#         c = np.eye(n)
#         for i in range(walk_length):
#             c = (1 - restart_p) * np.dot(c, p) + restart_p * np.eye(n)
#         return squareform(pdist(c,metric='cityblock'))
#     else:
#         pi = degree / degree.sum()
#         return squareform(pdist(inv(np.eye(n) - p - pi.T),metric='cityblock'))

In [3]:
def DSD_calculator(adjacency, walk_length, restart_p):
    """
    adjacency - adjacency matrix represented as a numpy array
                assumes graph is fully connected.
    walk_length - the length of random walks used to calculate DSD
                  if walk_length = -1, then calculate DSD at convergence
    restart_p - the restart probability
        if p = 0, then it's a traditional random walk
    returns DSD matrix represented as a numpy array
    """
    adjacency = np.asmatrix(adjacency)
    n = adjacency.shape[0]
    degree = adjacency.sum(axis=1)
    p = adjacency / degree
    if walk_length >= 0:
        c = np.eye(n)
        for i in range(walk_length):
            c = (1 - restart_p) * np.dot(c, p) + restart_p * np.eye(n)
        return squareform(pdist(c,metric='cityblock'))
    else:
        pi = degree / degree.sum()
        return squareform(pdist(inv(np.eye(n) - p - pi.T),metric='cityblock'))

# Load Dataset & Construct Graph

In [4]:
edges_df = pd.read_csv("data/git_web_ml/musae_git_edges.csv")
nodes_df = pd.read_csv("data/git_web_ml/musae_git_target.csv")
# G_github = nx.from_pandas_edgelist(edges_df, source="id_1", target='id_2')
# print('-----------------------')
# print('Original Graph:')
# print(G_github)
# # print(nx.info(G_github))

# sample_num = 5000
# sample_nodes = random.choices(list(G_github.nodes()), k=sample_num)
# G = G_github.subgraph(sample_nodes)
# print('-----------------------')
# print('Picked graph with 3000 nodes:')
# print(G)
# # print(nx.info(G))
# largest_cc = max(nx.connected_components(G), key=len)
# G = G.subgraph(largest_cc)
# print('-----------------------')
# print('Fully Connected smaller graph:')
# print(G)
# # print(nx.info(G))
# G_github = G
# # print(G_github.nodes())
# G = StellarGraph.from_networkx(G_github)

In [5]:
G = pickle.load(open('graph.pickle10k', 'rb'))
print(G)

Graph with 6069 nodes and 16162 edges


In [6]:
# Calculate DSD
# edges = nx.to_numpy_array(G_github)
edges = nx.to_numpy_array(G)
DSD = DSD_calculator(edges, 20, 0)

In [7]:
test = DSD
rows = len(DSD)
columns = len(DSD[0])
for i in range(rows):
    for j in range(columns):
        if DSD[i][j] == 0:
            test[i][j] = DSD[i][j]
        else:
            test[i][j] = 1.0 / DSD[i][j]

print(len(test))
node_list = list(G.nodes())
nodes_dftest = nodes_df
source_list = []
target_list = []
weight_list = []

for edges in G.edges():
    node1= edges[0]
    node2 = edges[1]
    index1 = node_list.index(node1)
    index2 = node_list.index(node2)
    weight = test[index1][index2]
    source_list.append(node1)
    target_list.append(node2)
    weight_list.append(weight)
    
weighted_edges = pd.DataFrame(source_list, columns=['source'])
weighted_edges['target'] = target_list
weighted_edges['weight'] = weight_list
G = StellarGraph(edges=weighted_edges)
# print(G.info())

6069


In [8]:
print(nodes_df.head())

   id          name  ml_target
0   0        Eiryyy          0
1   1    shawflying          0
2   2   JpMCarrilho          1
3   3     SuhwanCha          0
4   4  sunilangadi2          1


# Random Walk

In [9]:
rw = BiasedRandomWalk(G)

walks = rw.run(
    nodes=G.nodes(),  # root nodes
    length=80,  # maximum length of a random walk
    n=10,  # number of random walks per root node
    p=0.5,  # Defines (unormalised) probability, 1/p, of returning to source node
    q=2,  # Defines (unormalised) probability, 1/q, for moving away from source node
    weighted=True,  # for weighted random walks
    seed=42,  # random seed fixed for reproducibility
)

In [10]:
print(walks[110])

[27, 26124, 18937, 30524, 22321, 35006, 22321, 35372, 22321, 2806, 22321, 3647, 22321, 8065, 27803, 25811, 27803, 25908, 8458, 22650, 8458, 13284, 24843, 13284, 19789, 33045, 19789, 13284, 32647, 31944, 32647, 31944, 32647, 31944, 32647, 31944, 32647, 31944, 32647, 31944, 32647, 21957, 32647, 21957, 18507, 36099, 37087, 97, 37087, 21957, 18507, 32960, 18507, 21148, 28449, 6631, 28449, 10416, 35633, 31380, 35633, 6645, 9168, 29755, 37441, 29755, 37441, 33740, 27803, 7756, 23243, 7756, 27803, 23051, 18014, 27803, 34942, 27803, 10178, 18163]


# Node Embedding

In [11]:
str_walks = [[str(n) for n in walk] for walk in walks]

model = Word2Vec(str_walks, vector_size = 16, window=10, min_count=1, sg=1, workers=4)

In [12]:
emb_df = (
    pd.DataFrame(
        [model.wv.get_vector(str(n)) for n in G.nodes()],
        index = G.nodes()
    )
)

In [13]:
test = emb_df.merge(
    nodes_df[['id', 'ml_target']].set_index('id'),
    left_index = True,
    right_index = True
)
print(test.head())

              0         1         2         3         4         5         6  \
5      0.023817  0.047018 -0.386038  0.468563 -0.300095 -0.366555 -0.184568   
6     -0.488567  0.098497 -0.109907  0.565695 -0.331591 -0.374777  0.051145   
32774 -0.574875 -1.275952 -0.901271  0.541054  0.520959  0.645740 -0.291264   
32781  0.650406 -0.411459  0.153719  1.628315 -1.328040  0.319690  0.419854   
32782  0.238292 -0.124377 -0.495786  0.052395 -0.327776  0.152868 -0.395207   

              7         8         9        10        11        12        13  \
5      0.471268  0.035068 -0.229809  0.497745  0.114748 -0.299116  0.120563   
6      0.830111  0.362442  0.145871  0.566032  0.879432 -0.489558  0.526538   
32774  0.339795  1.656167  0.067100 -0.069274  0.610459 -0.252049 -1.055325   
32781 -1.185130  0.537332 -0.487745 -0.607852  0.311871 -0.062573 -0.240611   
32782  0.520561 -0.313204 -0.763758  0.388737  0.073375 -0.688577  0.196944   

             14        15  ml_target  
5      0.76

# Train Classifier

In [14]:
x = emb_df.values
y = test['ml_target'].values

x_train, x_test, y_train, y_test = train_test_split(
    x, 
    y,
    test_size = 0.7
)

score_list = []

for x in range(10):
    # GBC classifier
    clf = GradientBoostingClassifier(random_state=x)

    # train the model
    clf.fit(x_train, y_train)

    # evaluate Classifier
    print(clf.score(x_test, y_test))
    score_list.append(clf.score(x_test, y_test))

# # GBC classifier
# clf = GradientBoostingClassifier()

# # train the model
# clf.fit(x_train, y_train)

# # evaluate Classifier
# print(clf.score(x_test, y_test))
print(score_list)

0.8330587589236683
0.8325096101043383
0.8330587589236683
0.8319604612850082
0.8319604612850082
0.8330587589236683
0.8330587589236683
0.8330587589236683
0.8319604612850082
0.8325096101043383
[0.8330587589236683, 0.8325096101043383, 0.8330587589236683, 0.8319604612850082, 0.8319604612850082, 0.8330587589236683, 0.8330587589236683, 0.8330587589236683, 0.8319604612850082, 0.8325096101043383]


In [15]:
def DSD_calculator(adjacency, walk_length, restart_p):
    """
    adjacency - adjacency matrix represented as a numpy array
                assumes graph is fully connected.
    walk_length - the length of random walks used to calculate DSD
                  if walk_length = -1, then calculate DSD at convergence
    restart_p - the restart probability
        if p = 0, then it's a traditional random walk
    returns DSD matrix represented as a numpy array
    """
    adjacency = np.asmatrix(adjacency)
    n = adjacency.shape[0]
    degree = adjacency.sum(axis=1)
    p = adjacency / degree
    if walk_length >= 0:
        c = np.eye(n)
        for i in range(walk_length):
            c = (1 - restart_p) * np.dot(c, p) + restart_p * np.eye(n)
        return squareform(pdist(c,metric='cityblock'))
    else:
        pi = degree / degree.sum()
        return squareform(pdist(inv(np.eye(n) - p - pi.T),metric='cityblock'))

In [16]:
X = np.array([[0,2,3], [1,0,3], [3,4,0]])
dist_matrix = pdist(X, metric = 'cityblock')
# dist_matrix = pdist(X)
# print(dist_matrix)
print(squareform(dist_matrix))

[[0. 3. 8.]
 [3. 0. 9.]
 [8. 9. 0.]]
